# Imports

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [19]:
target = train['Interest_Rate']
train = train.drop('Interest_Rate', axis=1)

# Pre-processing

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164309 entries, 0 to 164308
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Loan_ID                  164309 non-null  int64  
 1   Loan_Amount_Requested    164309 non-null  object 
 2   Length_Employed          156938 non-null  object 
 3   Home_Owner               138960 non-null  object 
 4   Annual_Income            139207 non-null  float64
 5   Income_Verified          164309 non-null  object 
 6   Purpose_Of_Loan          164309 non-null  object 
 7   Debt_To_Income           164309 non-null  float64
 8   Inquiries_Last_6Mo       164309 non-null  int64  
 9   Months_Since_Deliquency  75930 non-null   float64
 10  Number_Open_Accounts     164309 non-null  int64  
 11  Total_Accounts           164309 non-null  int64  
 12  Gender                   164309 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 16.3+ MB


Columns having empty values : Home_Owner, Length_Employed, Annual_income, Months_Since_deliquency

In [21]:
train

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164304,10164305,"20,725",10+ years,Rent,NaN,VERIFIED - income source,credit_card,13.56,0,NaN,13,16,Male
164305,10164306,"6,000",8 years,Own,NaN,VERIFIED - income source,small_business,9.12,0,36.0,13,30,Male
164306,10164307,"20,250",10+ years,Mortgage,NaN,VERIFIED - income,credit_card,18.17,1,NaN,17,30,Male
164307,10164308,"10,000",4 years,Rent,NaN,not verified,major_purchase,18.75,1,48.0,16,62,Female


In [22]:
train = train.drop('Loan_ID', axis=1)
test = test.drop('Loan_ID', axis=1)

In [23]:
train['Loan_Amount_Requested'] = train['Loan_Amount_Requested'].str.replace(',','')
test['Loan_Amount_Requested'] = test['Loan_Amount_Requested'].str.replace(',','')

In [24]:
train['Annual_Income'].fillna(train['Annual_Income'].mean(), inplace=True)
test['Annual_Income'].fillna(test['Annual_Income'].mean(), inplace=True)

In [25]:
train['Home_Owner'].fillna("Mortgage", inplace=True)
test['Home_Owner'].fillna("Mortgage", inplace=True)

In [26]:
train['Length_Employed'].fillna('10+ years', inplace=True)
test['Length_Employed'].fillna('10+ years', inplace=True)

In [27]:
train['Months_Since_Deliquency'].fillna('0', inplace=True)
test['Months_Since_Deliquency'].fillna('0', inplace=True)

In [28]:
combine = [train, test]

for dataset in combine:
    dataset['Loan_Amount_Requested'] = dataset['Loan_Amount_Requested'].astype(int)
    dataset['Home_Owner'] = dataset['Home_Owner'].replace(['Own','Other','None'],'Other')
    dataset['Home_Owner'] = dataset['Home_Owner'].map( {'Other': 2, 'Rent':1, 'Mortgage': 0} ).astype(int)
    dataset['Purpose_Of_Loan'] = dataset['Purpose_Of_Loan'].replace(['credit_card','home_improvement','other','small_business','vacation','wedding','Major','major_purchase','car','house','medical','renewable_energy','moving','educational'],'Non debt_consolidation')
    dataset['Purpose_Of_Loan'] = dataset['Purpose_Of_Loan'].map( {'debt_consolidation': 1, 'Non debt_consolidation': 0} ).astype(int)
    dataset['Length_Employed'] = dataset['Length_Employed'].replace(['7 years','8 years','9 years','6 years','5 years'],'5+ years')
    dataset['Length_Employed'] = dataset['Length_Employed'].replace(['< 1 year','1 year','2 years','3 years','4 years'],'Others')
    dataset['Length_Employed'] = dataset['Length_Employed'].map( {'Others': 2, '5+ years' : 1, '10+ years': 0} ).astype(int)
    dataset['Gender'] = dataset['Gender'].map( {'Female': 1, 'Male': 0} ).astype(int)
    dataset['Income_Verified'] = dataset['Income_Verified'].map( {'not verified': 2, 'VERIFIED - income':1,'VERIFIED - income source': 0} ).astype(int)
    dataset['Loan_Amount_Requested'] = dataset['Loan_Amount_Requested'].fillna(0)
    dataset['Loan_Amount_Requested'] = dataset['Loan_Amount_Requested'].astype(int)

In [29]:
train

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,7000,2,1,68000.000000,2,0,18.37,0,0,9,14,1
1,30000,2,0,73331.159434,1,1,14.93,0,17,12,24,1
2,24725,1,0,75566.400000,0,1,15.88,0,0,12,16,0
3,16000,2,0,56160.000000,0,1,14.34,3,0,16,22,0
4,17000,1,2,96000.000000,0,1,22.17,1,0,19,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...
164304,20725,0,1,73331.159434,0,0,13.56,0,0,13,16,0
164305,6000,1,2,73331.159434,0,0,9.12,0,36,13,30,0
164306,20250,0,0,73331.159434,1,0,18.17,1,0,17,30,0
164307,10000,2,1,73331.159434,2,0,18.75,1,48,16,62,1


# Models

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(train, target, test_size=0.2)

**Logistic Regression**

In [34]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)

In [35]:
accuracy, precision, recall, f1 = get_metrics(Y_pred, Y_test)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.455, precision = 0.586, recall = 0.455, f1 = 0.502


**Random Forest**

In [36]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=5)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)

In [37]:
accuracy, precision, recall, f1 = get_metrics(Y_pred, Y_test)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.496, precision = 0.631, recall = 0.496, f1 = 0.547


**AdaBoost**

In [38]:
# Fitting Ada-boost to the Training set
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
ADA = AdaBoostClassifier(base_estimator=None, n_estimators=420, learning_rate=1.0)
ADA.fit(X_train, Y_train)
y_pred = ADA.predict(X_test)

In [39]:
#Let's see how our model performed
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.53      0.25      0.33      6856
           2       0.49      0.63      0.55     14061
           3       0.57      0.54      0.56     11945

    accuracy                           0.52     32862
   macro avg       0.53      0.47      0.48     32862
weighted avg       0.52      0.52      0.51     32862



**Gradient Boosting**

In [40]:
# Fitting GradientBoosting to the Training set
from sklearn.neighbors import KNeighborsClassifier
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=400, subsample=1.0,
                                   min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                   max_depth=8,
                                   init=None, random_state=None, max_features=None, verbose=0)
GB.fit(X_train, Y_train)
y_pred = GB.predict(X_test)

In [41]:
#Let's see how our model performed
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.49      0.28      0.36      6856
           2       0.49      0.60      0.54     14061
           3       0.57      0.55      0.56     11945

    accuracy                           0.52     32862
   macro avg       0.51      0.48      0.49     32862
weighted avg       0.52      0.52      0.51     32862



**Search for better parameters**

In [44]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [45]:
adaHyperParams = {'n_estimators': [10,50,100,200,420]}
gridSearchAda = RandomizedSearchCV(estimator=ADA, param_distributions=adaHyperParams, n_iter=5,scoring='f1_weighted', cv=None, verbose=2).fit(X_train, Y_train)
gridSearchAda.best_params_, gridSearchAda.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=10 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................................. n_estimators=10, total=   2.5s
[CV] n_estimators=10 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


[CV] .................................. n_estimators=10, total=   2.3s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   2.3s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   2.1s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   2.1s
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=  10.6s
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=  10.4s
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=  11.3s
[CV] n_estimators=50 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 12.9min finished


({'n_estimators': 420}, 0.5110383506601014)

In [ ]:
# gbHyperParams = {'loss' : ['deviance'],
#                  'n_estimators': randint(10, 500),
#                  'max_depth': randint(1,10)}
# gridSearchGB = RandomizedSearchCV(estimator=GB, param_distributions=gbHyperParams, n_iter=10,
#                                    scoring='f1_weighted', cv=None, verbose=2).fit(X_train, Y_train)
# gridSearchGB.best_params_, gridSearchGB.best_score_

**Ensemble Model - GradientBoost and AdaBoost**

In [42]:
from sklearn.ensemble import VotingClassifier
votingMod = VotingClassifier(estimators=[('gb', GB), 
                                         ('ada', ADA)], voting='soft',weights=[2,1])
votingMod = votingMod.fit(X_train, Y_train)

In [43]:
test_labels=votingMod.predict_proba(np.array(X_test))[:,1]
votingMod.score(X_test, Y_test)

0.5215142109427302